# PREDICTION WEB APPLICATION

### Importing liabriess 

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sb
import os
import glob
import json 
import requests
from bs4 import BeautifulSoup
%matplotlib inline

#### Creating function for Downloading Data For Training and Testing The Model

In [2]:
# os.listdir('./Prediction_train&test_Data')
def load_data(url):
    
    folderName = './Prediction_train&test_Data/'
    fileName = 'league_data.json'
    path = os.path.join(folderName, fileName)

    if os.path.exists(path):
        files = glob.glob('./Prediction_train&test_Data/*')
        for f in files:
            os.remove(f)

    response = requests.get(url)
    # print(path)

    with open(path, 'wb') as outfile:
        outfile.write(response.content)
     

        
url = 'https://vsmobile.bet9ja.com/shopadmin/standings_view/league_data.php?pid=14001'
load_data(url)

#### Reading Json Files 

In [3]:
folderName = './Prediction_train&test_Data/'
fileName = 'league_data.json'
path = os.path.join(folderName, fileName)

with open(path, 'r') as outfile:
    json_obj = json.load(outfile)

In [4]:
clubes = json_obj['teams']
matches = json_obj['matches']
teams = []
resulte = []
wonmarke = []

count = 0
for key, val in matches.items():
    
    teams.append({key: []})
    resulte.append({key: []})
    wonmarke.append({key: []})
    
    for x in val:
        
        teams[count][key].append([val[x]['teamA'], val[x]['teamB']])
        resulte[count][key].append(val[x]['result'])
        wonmarke[count][key].append(val[x]['wonmarkets'])
        
    count +=1

In [5]:
# json_obj
# teams
# resulte
# wonmarke

#### Wranggling Data into Data Base

- Data
    - columns:
        - teams
        - winners(0 home, 1 draw, 2 away)



In [6]:
# resulte

In [7]:
# wonmarke
# teams

In [8]:
# events = []
events = {}
teams_index = {}
for x in range(len(teams)):
    for y, obj in teams[x].items():
        events[y] = []
        teams_index[y] = []
        for i in obj:
            teamA = clubes[i[0]]['fifa']
            teamB = clubes[i[1]]['fifa']
            events[y].append(f'{teamA}-{teamB}')
            teams_index[y].append('-'.join(i))

In [9]:
score = {}
for x in range(len(wonmarke)):
    for y, string in wonmarke[x].items():
        score[y] = []
        for j in string:
            resulte_win = j.split(',')[0]
            score[y].append(resulte_win)

In [10]:
# score
# events
socore_dataframe = pd.DataFrame(score)
events_dataframe = pd.DataFrame(events)
teams_index_dataframe = pd.DataFrame(teams_index)

In [11]:
socore_dataframe.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,Home,Home,Away,Draw,Home,Home,Away,Away,Home,Away,Draw,Home,Home,Draw,Home
1,Draw,Draw,Home,Away,Home,Away,Away,Home,Home,Home,Draw,Home,Away,Draw,Away
2,Draw,Away,Draw,Home,Away,Draw,Away,Draw,Home,Draw,Home,Draw,Away,Draw,Home
3,Home,Away,Home,Draw,Away,Home,Home,Draw,Away,Away,Away,Away,Home,Home,Home
4,Home,Home,Away,Draw,Draw,Home,Draw,Away,Draw,Home,Home,Draw,Home,Draw,Home


In [12]:
events_dataframe.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,WOL-BRN,WHU-NOR,NOR-EVE,BRN-WAT,BUR-ARS,MNC-ASV,ASV-TOT,NOR-ASV,BRI-EVE,NOR-LEI,NWC-ARS,ASV-BUR,LEE-WOL,MNC-LEE,CRY-ASV
1,ASV-SOU,MNU-LEI,LIV-ARS,WHU-LIV,LEI-WHU,ARS-CHE,NOR-BRN,ARS-CRY,LIV-NOR,ASV-LIV,WAT-EVE,WHU-NWC,LEI-BRN,BRN-ARS,SOU-EVE
2,TOT-WHU,SOU-LIV,ASV-WHU,WOL-NOR,ASV-WOL,SOU-BRI,LIV-MNC,TOT-LIV,LEE-SOU,ARS-LEE,MNU-SOU,TOT-BRI,MNU-WHU,EVE-MNU,CHE-LEI
3,BRI-CRY,WAT-BRI,LEI-SOU,EVE-ASV,CRY-NWC,EVE-LEI,CHE-WHU,WOL-BUR,CRY-WHU,SOU-NWC,CRY-WOL,LIV-LEI,SOU-ARS,LEI-BUR,LEE-TOT
4,MNC-EVE,ARS-ASV,CHE-NWC,SOU-BUR,LIV-EVE,NWC-WAT,LEE-NWC,WHU-BRI,LEI-TOT,WHU-WAT,CHE-TOT,MNC-CRY,BRI-NOR,NOR-CRY,MNU-WOL


In [13]:
teams_index_dataframe.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,525-972,58-66,66-49,972-70,64-48,46-60,60-51,66-60,71-49,66-63,53-48,60-64,827-525,46-827,55-60
1,60-52,50-63,45-48,58-45,63-58,48-47,66-972,48-55,45-66,60-45,70-49,58-53,63-972,972-48,52-49
2,51-58,52-45,60-58,525-66,60-525,52-71,45-46,51-45,827-52,48-827,50-52,51-71,50-58,49-50,47-63
3,71-55,70-71,63-52,49-60,55-53,49-63,47-58,525-64,55-58,52-53,55-525,45-63,52-48,63-64,827-51
4,46-49,48-60,47-53,52-64,45-49,53-70,827-53,58-71,63-51,58-70,47-51,46-55,71-66,66-55,50-525


In [14]:
def melt_table(dataframe):
    dataframe['index'] = 1
    values_vars = [str(x) for x in range(1, dataframe.shape[1])]
    resulte = pd.melt(dataframe, id_vars=['index'], value_vars=values_vars)
    return resulte

In [15]:
new_events_df = melt_table(events_dataframe).rename(columns={'value': 'events', 'variable': 'weeks'})[['events','weeks']]
new_score_df = melt_table(socore_dataframe).rename(columns={'value': 'score', 'variable': 'weeks'})[['score','weeks']]
teams_index_df = melt_table(teams_index_dataframe).rename(columns={'value': 'teams_index', 'variable': 'weeks'})[['teams_index','weeks']]

In [16]:
new_events_df.head()

,events,weeks
0,WOL-BRN,1
1,ASV-SOU,1
2,TOT-WHU,1
3,BRI-CRY,1
4,MNC-EVE,1


In [17]:
new_score_df.head()

,score,weeks
0,Home,1
1,Draw,1
2,Draw,1
3,Home,1
4,Home,1


In [18]:
teams_index_df.head()

,teams_index,weeks
0,525-972,1
1,60-52,1
2,51-58,1
3,71-55,1
4,46-49,1


In [19]:
events_score_df = pd.concat([new_events_df, new_score_df, teams_index_df], axis=1, join="inner").iloc[:, np.r_[0:2, 2, 4]] # [['events', 'score', 'teams_index']]

In [20]:
events_score_df.head(6)

,events,weeks,score,teams_index
0,WOL-BRN,1,Home,525-972
1,ASV-SOU,1,Draw,60-52
2,TOT-WHU,1,Draw,51-58
3,BRI-CRY,1,Home,71-55
4,MNC-EVE,1,Home,46-49
5,BUR-LEE,1,Draw,64-827


#### Saving Data Into CSV file

In [21]:
events_score_df.to_csv('./Prediction_train&test_Data/premier_leage_data.csv', index=False)